In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
import json
from io import BytesIO
import os
import re
from itertools import chain

from google.cloud import bigquery
from google.oauth2 import service_account

from datetime import datetime, timedelta
from dateutil import parser
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# from openpyxl import Workbook
# from openpyxl.utils.dataframe import dataframe_to_rows
# from openpyxl.styles import Font

import base64

# import feedparser

import requests
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# InsecureRequestWarning 경고 무시
warnings.simplefilter('ignore', InsecureRequestWarning)

##### 구글빅쿼리 설정

In [2]:
KEY_PATH = ".config/"
servicekey_path = KEY_PATH + "serviceKey.json" ## 빅쿼리 외 다른 API 활용 위해
bigquerykey_path = KEY_PATH + "mido-project-426906-31b49963ac97.json"

warnings.filterwarnings("ignore")

In [3]:
# BigQuery 클라이언트 생성 함수
def create_bigquery_client(key_path):
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    return client

In [4]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    # BigQuery 클라이언트 객체 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [5]:
def get_dataframe_from_bigquery(dataset_id, table_id, key_path):
    # BigQuery 클라이언트 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 테이블 데이터를 DataFrame으로 변환
    df = client.list_rows(table_ref).to_dataframe()

    return df

In [6]:
# 오늘 날짜
today = datetime.today().strftime('%Y%m%d')

# 어제 날짜 계산
ytday = datetime.today() - timedelta(days=1)

# 만약 어제가 토요일(5) 또는 일요일(6)이라면, 그 전주 금요일로 변경
if ytday.weekday() == 5:  # 토요일
    ytday -= timedelta(days=1)
elif ytday.weekday() == 6:  # 일요일
    ytday -= timedelta(days=2)

# 'YYYYMMDD' 형식으로 변환
ytday = ytday.strftime('%Y%m%d')

In [7]:
create_bigquery_client(bigquerykey_path).list_tables(f"{'mido-project-426906'}.{'g2b'}")

### 나라장터 종합쇼핑몰 납품 상세 내역

In [8]:
# API 엔드포인트 및 쿼리 파라미터 설정
url = 'https://apis.data.go.kr/1230000/ShoppingMallPrdctInfoService06/getDlvrReqDtlInfoList01'
params = {
    'ServiceKey': 'mQ9FwLgvOY3RCjUPiBiGw0ZjjfJW1G5IWVopKvRrp/+e+FCQR8ClAQ2iNB0DFLVZbCjRY+QVE7/Wy8BbCF4O7w==',
    'pageNo': '1',
    'numOfRows': '100',
    'type': 'json',
    'inqryDiv': '1',
    'inqryBgnDate': today[:-2]+'01', ## 해당월
    'inqryEndDate': today, ## 오늘날짜
    'prdctClsfcNoNm': '인조잔디'
}


# 모든 데이터를 저장할 빈 리스트 초기화
all_items = []

# 페이지를 바꿔가며 데이터를 가져오기
page_no = 1
while True:
    params['pageNo'] = str(page_no)
    response = requests.get(url, params=params)
    data = response.json()
    
    # 현재 페이지의 items 가져오기
    items = data['response']['body']['items']
    
    # items가 비어있으면 반복 종료
    if not items:
        break
    
    # 현재 페이지의 items를 all_items에 추가
    all_items.extend(items)
    
    # 다음 페이지로 이동
    page_no += 1

# 모든 items 데이터를 데이터프레임으로 변환
shop_detail_df = pd.DataFrame(all_items)

# 컬럼명을 한글로 변경
column_mapping = {
    'dlvrReqNo': '납품요구번호',
    'dlvrReqChgOrd': '납품요구변경차수',
    'dlvrReqRcptDate': '납품요구접수일자',
    'prdctSno': '물품순번',
    'prdctClsfcNo': '물품분류번호',
    'prdctClsfcNoNm': '품명',
    'dtilPrdctClsfcNo': '세부물품분류번호',
    'dtilPrdctClsfcNoNm': '세부품명',
    'prdctIdntNo': '물품식별번호',
    'prdctIdntNoNm': '품목',
    'prdctUprc': '단가',
    'prdctUnit': '단위',
    'prdctQty': '수량',
    'prdctAmt': '금액',
    'dlvrTmlmtDate': '납품기한일자',
    'cntrctCnclsStleNm': '계약구분',
    'exclcProdctYn': '우수제품여부',
    'optnDivCdNm': '옵션구분',
    'dminsttCd': '수요기관코드',
    'dminsttNm': '수요기관명',
    'dmndInsttDivNm': '수요기관구분',
    'dminsttRgnNm': '수요기관지역명',
    'corpNm': '업체명',
    'fnlDlvrReqYn': '최종납품요구여부',
    'incdecQty': '증감납품요구수량',
    'incdecAmt': '증감납품요구금액',
    'cntrctCorpBizno': '업체사업자등록번호',
    'dlvrReqNm': '납품요구건명',
    'cntrctNo': '계약번호',
    'cntrctChgOrd': '계약변경차수',
    'masYn': '다수공급자계약여부',
    'cnstwkMtrlDrctPurchsObjYn': '공사용자재직접구매대상여부',
    'smetprCmptProdctYn': '중소기업자간경쟁제품여부',
    'IntlCntrctDlvrReqDate': '최초납품요구접수일자',
    'dlvrReqQty': '납품요구수량',
    'dlvrReqAmt': '납품요구금액',
    'corpEntrprsDivNmNm': '업체기업구분명',
    'brnofceNm': '납품요구지청명'
}

shop_detail_df.rename(columns=column_mapping, inplace=True)

# 필터링
shop_detail_df = shop_detail_df[shop_detail_df['품명'] == '인조잔디']
# shop_detail_df = shop_detail_df[shop_detail_df['최종납품요구여부'] == 'Y'].reset_index(drop=True)

# 숫자형 컬럼 변환
numeric_columns = ['단가', '수량', '금액', '납품요구수량', '납품요구금액', '증감납품요구수량', '증감납품요구금액']
shop_detail_df[numeric_columns] = shop_detail_df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# 최종데이터셋
shop_detail_df = shop_detail_df[['납품요구번호', '납품요구변경차수', '물품순번', '수요기관명', '납품요구접수일자', '납품요구건명', '업체명', '물품분류번호', '품명', '세부물품분류번호', '세부품명', 
                                 '물품식별번호', '품목', '단가', '단위', '수량', '금액', '납품기한일자', '계약구분', '계약번호', '계약변경차수', '다수공급자계약여부', '공사용자재직접구매대상여부', 
                                 '중소기업자간경쟁제품여부', '우수제품여부', '옵션구분', '최종납품요구여부', '최초납품요구접수일자', '납품요구수량',  '납품요구금액', '증감납품요구수량', '증감납품요구금액', 
                                 '수요기관코드', '수요기관구분', '수요기관지역명', '업체사업자등록번호', '업체기업구분명', '납품요구지청명']]

In [9]:
# 빅쿼리 적재
save_dataframe_to_bigquery(shop_detail_df,'g2b','shop_detail_df_0'+today[:-2],bigquerykey_path)

Data inserted into table shop_detail_df_0202408 successfully.


In [10]:
# 전체 종합쇼핑몰 데이터 가져오기
tables = create_bigquery_client(bigquerykey_path).list_tables(f"{'mido-project-426906'}.{'g2b'}")
table_list = [table.table_id for table in tables if 'shop_detail_df' in table.table_id and 'all' not in table.table_id and 'total' not in table.table_id and 'view' not in table.table_id]

shop_df_list = []
for tb_nm in table_list:
    data = get_dataframe_from_bigquery('g2b', tb_nm,bigquerykey_path)
    shop_df_list.extend(data.to_dict('records'))

In [11]:
# total_shop_df = get_dataframe_from_bigquery('g2b', 'shop_detail_df_total', bigquerykey_path)
all_shop_df = pd.DataFrame(shop_df_list)

In [12]:
# 빅쿼리 적재
save_dataframe_to_bigquery(all_shop_df,'g2b','shop_detail_df_all',bigquerykey_path)

Data inserted into table shop_detail_df_all successfully.


### 나라장터 종합쇼핑몰 품목 정보 조회

In [13]:
# API 엔드포인트 및 쿼리 파라미터 설정
years= ['2022','2023','2024']
all_items = []
for year in years:
    url = 'https://apis.data.go.kr/1230000/ShoppingMallPrdctInfoService06/getShoppingMallPrdctInfoList01'
    params = {
        'ServiceKey': "mQ9FwLgvOY3RCjUPiBiGw0ZjjfJW1G5IWVopKvRrp/+e+FCQR8ClAQ2iNB0DFLVZbCjRY+QVE7/Wy8BbCF4O7w==",
        'pageNo': '1',
        'numOfRows': '100',
        'type': 'json',
        'inqryDiv': '1',
        'inqryBgnDate': year + '0101',#today[:-2]+'01', ## 해당월
        'inqryEndDate': year + '1231',#today, ## 오늘날짜
        'prdctClsfcNoNm': '인조잔디',
        # 'exclcProdctYn': 'Y',
        # 'masYn': 'Y',
        # 'prodctCertYn': 'Y',
    }


    # 모든 데이터를 저장할 빈 리스트 초기화
    # all_items = []

    # 페이지를 바꿔가며 데이터를 가져오기
    page_no = 1
    while True:
        params['pageNo'] = str(page_no)
        response = requests.get(url, params=params)
        data = response.json()
        
        # 현재 페이지의 items 가져오기
        items = data['response']['body']['items']
        
        # items가 비어있으면 반복 종료
        if not items:
            break
        
        # 현재 페이지의 items를 all_items에 추가
        all_items.extend(items)
        
        # 다음 페이지로 이동
        page_no += 1

# 모든 items 데이터를 데이터프레임으로 변환
shop_prodinfo_df = pd.DataFrame(all_items)

In [14]:
save_dataframe_to_bigquery(shop_prodinfo_df,'g2b','shop_prod_info',bigquerykey_path)

Data inserted into table shop_prod_info successfully.


### 지자체 예산서 링크

In [15]:
# API 엔드포인트 및 기본 파라미터 설정
url = "https://www.lofin365.go.kr/lf/hub/BUDLK"

api_key = "UPDXX1000317620240618111320UXQEE"

# 요청 헤더
headers = {
    "Authorization": f"Bearer {api_key}"
}

params = {
    "Key": api_key,
    "Type": "json",
    "pIndex": 1,
    "pSize": 1000,
    "fyr": 2024,
}

In [16]:
response = requests.get(url, headers=headers, params=params, verify=False)
data = response.json()

In [17]:
budget_link_df = pd.DataFrame(data['BUDLK'][1]['row'])
budget_link_df.columns = ['기준년도', '지역명', '자치단체코드', '자치단체명', '링크명', 'URL']

In [18]:
save_dataframe_to_bigquery(budget_link_df,'budget','budget_link_df',bigquerykey_path)

Data inserted into table budget_link_df successfully.


### 지자체 사업별 예산

In [19]:
# API 엔드포인트 및 기본 파라미터 설정
url = "https://www.lofin365.go.kr/lf/hub/QWGJK"

api_key = "TMGNU1000317620240618111419PWNQE"

# 요청 헤더
headers = {
    "Authorization": f"Bearer {api_key}"
    }

# 데이터 수집을 위한 빈 리스트 생성
all_data = []
page = 1
page_size = 1000

while True:
    # 요청 파라미터
    params = {
        "Key": api_key,
        "Type": "json",
        "pIndex": page,
        "pSize": page_size,
        "fyr": today[:4],
        "exe_ymd": today # ex) "exe_ymd": '20240628'
    }

    # API 요청 보내기 (certifi를 사용하여 SSL 인증서 검증)
    response = requests.get(url, headers=headers, params=params, verify=False)

    # 응답 확인
    if response.status_code == 200:
        data = response.json()

        # 데이터를 리스트에 추가
        if data:
            all_data.extend(data['QWGJK'][1]['row'])
            # 다음 페이지로 이동
            page += 1
        else:
            # 더 이상 데이터가 없으면 중지
            break
    else:
        print(f"Failed to fetch data for page {page}: {response.status_code}")
        break

# # 수집된 모든 데이터 출력
# print(all_data)

In [ ]:
bid_budget_df = pd.DataFrame(all_data)

In [ ]:
column_mapping = {'acnt_dv_nm':'회계구분명',
                  'dept_cd':'부서코드',
                  'dbiz_cd':'세부사업코드',
                  'dbiz_nm':'세부사업명',
                  'exe_ymd':'집행일자',
                  'bdg_cash_amt':'예산현액',
                  'bdg_ntep':'국비',
                  'capep':'시도비',
                  'sggep':'시군구비',
                  'etc_amt':'기타',
                  'ep_amt':'지출액',
                  'cpl_amt':'편성액',
                  'fld_cd':'분야코드',
                  'fld_nm':'분야명',
                  'ane_part_cd':'부문코드',
                  'part_nm':'부문명',
                  'acnt_dv_cd':'회계구분코드',
                  'fyr':'회계연도',
                  'wa_laf_cd':'지역코드',
                  'wa_laf_hg_nm':'지역명',
                  'laf_cd':'자치단체코드',
                  'laf_hg_nm':'자치단체명',
                  'padm_laf_cd':'행정자치단체코드'}

In [ ]:
bid_budget_df.rename(columns=column_mapping,inplace=True)
bid_budget_df = bid_budget_df[['회계연도', '지역코드', '지역명', '자치단체코드', '자치단체명', '회계구분명', '세부사업코드', '세부사업명', '집행일자', '예산현액', '지출액', '편성액', '분야명', '부문명', '행정자치단체코드']] # '국비', '시도비', '시군구비', '기타'

In [ ]:
# 필요키워드1
need_kwd1 = '학교|초등학교|중학교|고등학교|대학교|다목적|운동장|종합운동장|공원|체육|체육공원|체육시설|체육센터|스포츠|스포츠센터|놀이시설|연병장|축구장|족구장|야구장|풋살장|배드민턴장|테니스장|게이트볼장|하키장'

# 필요키워드2
need_kwd2 = '개선|교체|보수|공사|구매|구입|설치|정비|조성'

# 필요키워드3
need_kwd3 = '인조잔디'

# 제외키워드
except_kwd = 'CCTV|cctv|LED|led|가로등|가속기|간접등|간판|개방|검진|경관|계단|골프장|관리|광고|교량|교실|교육|교차로|균열|그늘막|그림|급수관|급식|난방기|내진|냉방기|네트워크|노상|노점|농기계|단열|데이터|도로개설|도로방음벽|도로정비|도로확장|도서관|디자인|뚜껑|막구조물|맨홀|모노레일|모니터링|미끄럼|미세먼지|미술|방송|방수|방음|방음벽|배면도로|배수|버스|법률자문|벤치|벽화|보건|보도블럭|보상|보일러|보행자도로|복지|볼라드|분묘|불법|빗물받이|사용료|사워실|살포|생육환경|석면|선홈통|소프트웨어|수도|수로|수목표찰|수영장|스크린|스프링클러|승강기|시계탑|시스템|식수대|신호등|안심벨|안전|약수터|억제제|역량|연결도로|예방|예술|오염|옹벽|요양기관|운영|울타리|위생|위험수목|유아|육교|음성안내|음악|의자|인양기|인터넷|입학|자료|자전거|저장고|저장장치|전광판|전기|전신주|전통시장|정화|정화조|조도|조명|조형물|조화기|주차장|줄눈|진로진학|진입도로|창고|창출|처우|철거|취업|치안|카메라|카약|통학로|파고라|펜스|폐기물|폐열|포충기|풀장|풍차|하수|하수관로|하수도|하천|학습|행사|화장|화장실|활성화|횡단보도|휀스|흡연'

In [ ]:
bid_budget_df_filter = bid_budget_df[((bid_budget_df['세부사업명'].str.contains(need_kwd1) & 
                                  bid_budget_df['세부사업명'].str.contains(need_kwd2)) &
                                  ~bid_budget_df['세부사업명'].str.contains(except_kwd)) |
                                  bid_budget_df['세부사업명'].str.contains(need_kwd3)
                                  ].reset_index(drop=True)

In [ ]:
# 빅쿼리에 적재
save_dataframe_to_bigquery(bid_budget_df_filter,'budget','budget_df_0'+today,bigquerykey_path)

Data inserted into table budget_df_020240806 successfully.


### 교육청 예산

In [ ]:
# edu_df = pd.read_csv('C:/py_src/midoproject/data/2024년 교육청 본예산 정리.csv')
# edu_df.columns = ['시도','시군구','구분','과업명','금액','면적','예산집행']

In [ ]:
# save_dataframe_to_bigquery(edu_df,'edu','edu_budget',bigquerykey_path)

### 뉴스기사스크랩핑

#### 네이버뉴스

In [ ]:
# 네이버 API 인증 정보
client_id = 'AtyNtq218QgbEcjHkYAI'
client_secret = 'JDVrsZkBtZ'

# 검색 파라미터
keyword = ['학교', '초등학교', '중학교', '고등학교', '대학교', '다목적', '운동장', '종합운동장', '공원', '체육', '체육공원', '체육시설', '체육센터', 
           '스포츠', '스포츠센터', '놀이시설', '연병장', '축구장','족구장', '야구장', '풋살장', '배드민턴장', '테니스장','게이트볼장', '하키장']

display = 100  # 한 번에 가져올 검색 결과 출력 건수
sort = 'date'  # 최신순 정렬
max_results = 100  # 원하는 최대 검색 결과 건수


# 결과를 저장할 리스트
all_news = []

for kwd in tqdm(keyword):
    # 반복하여 API 요청 보내기
    for start in range(1, max_results + 1, display):
        url = f"https://openapi.naver.com/v1/search/news.json?query={kwd}&display={display}&sort={sort}&start={start}"
        headers = {
            'X-Naver-Client-Id': client_id,
            'X-Naver-Client-Secret': client_secret
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            news_data = response.json()
            all_news.extend(news_data['items'])
        else:
            print("Error Code:", response.status_code)
            break

100%|██████████| 25/25 [00:03<00:00,  8.24it/s]


In [ ]:
# 뉴스 기사 링크에서 제목과 본문을 가져오기 위한 함수
def get_article_content(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.find('meta', property='og:title')['content']
        content = soup.find('article', {'id': 'dic_area'}).get_text(strip=True)
        return title, content, url
    else:
        return None, None, url

# URL 리스트에서 제목과 본문 가져오기
def fetch_news_content(news_links):
    results = []
    for link in tqdm(news_links):
        title, content, url = get_article_content(link)
        if title and content:
            results.append({'title': title, 'content': content, 'link': url})
    return pd.DataFrame(results)

In [ ]:
# 네이버뉴스형식 기사 추출
all_news_df = pd.DataFrame(all_news)

drop_idx = []
for i in range(len(all_news_df['link'])):
    if 'n.news.naver.com' not in all_news_df['link'][i]:
            drop_idx = drop_idx + [i]

news_df = all_news_df.drop(drop_idx).reset_index(drop=True)
news_df = news_df.drop_duplicates(['link']).reset_index(drop=True)

In [ ]:
# 링크가 있는 데이터프레임의 컬럼을 리스트로 변환
news_links = news_df['link'].tolist()

# 뉴스 기사 내용 가져오기
news_links_df = fetch_news_content(news_links)

100%|██████████| 576/576 [01:59<00:00,  4.83it/s]


In [ ]:
naver_news_df = pd.merge(news_df,news_links_df,how='left',on='link')

In [ ]:
naver_news_df_fin = naver_news_df[['pubDate', 'title_y', 'content','originallink']]
naver_news_df_fin.columns = ['기사날짜', '제목', '본문', '원본링크']

In [ ]:
naver_news_df_fin = naver_news_df_fin.drop_duplicates() # 중복제거

# 날짜 형식 변경
naver_news_df_fin['기사날짜'] = naver_news_df_fin['기사날짜'].apply(lambda x: parser.parse(x).strftime('%Y-%m-%d'))
# naver_news_df_fin['기사날짜'] = naver_news_df_fin['기사날짜'].str.split().str[0]
naver_news_df_fin = naver_news_df_fin.sort_values(['기사날짜'],ascending=False).reset_index(drop=True)

In [ ]:
save_dataframe_to_bigquery(naver_news_df_fin, 'news', 'news_daily_0'+today, bigquerykey_path)

Data inserted into table news_daily_020240806 successfully.


### 인포21

In [ ]:
# 크롬 옵션 설정
chrome_options = Options()
# chrome_options.add_argument('--headless')  # 브라우저 창을 띄우지 않음
chrome_options.add_argument('--disable-gpu')

# 크롬 드라이버 서비스 설정
# service = Service(ChromeDriverManager().install())
service = Service()

# 웹 드라이버 초기화
driver = webdriver.Chrome(service=service, options=chrome_options)
time.sleep(2)

In [ ]:
# 웹페이지 접속
url = 'https://infose.info21c.net/info21c/bids/list/index?bidtype=con&bid_suc=bid&page=1&per-page=100'
driver.get(url)
time.sleep(2)

In [ ]:
# 로그인
driver.find_element(By.CSS_SELECTOR, '#id').send_keys('midoplus') ## ID입력
driver.find_element(By.CSS_SELECTOR, '#pass').send_keys('mido6235!!') ## password입력

driver.find_element(By.CSS_SELECTOR, '#login_btn').click()
time.sleep(2)

#### 입찰정보

In [ ]:
# bid_suc - bid (입찰)
# bidtype - con (공사) / ser (용역) / pur (구매)

# bid_suc - suc (낙찰)
# bidtype - con (공사) / ser (용역) / pur (구매)

In [ ]:
# 입찰
# 공사 - 공고명/공고번호	발주기관	추정가격	기초금액	개찰일	투찰마감	입력일	업종	지역
# 용역 - 공고명/공고번호	발주기관	추정가격	기초금액	개찰일	투찰마감	입력일	업종	지역
# 구매 - 공고명/공고번호	기초금액	개찰일	    업종	     ㅁ	   참가마감	  투찰마감

In [ ]:
# 낙찰
# 공사 - 공고명/공고번호	기초금액	예정가격	1순위투찰금액	1순위기초대비	개찰일	업종
# 용역 - 공고명/공고번호	발주기관	예정가격	예가/기초	1순위업체	개찰일	업종	지역
# 구매 - 공고명/공고번호	기초금액	개찰일	업종

In [ ]:
bid_info_dict={}

In [ ]:
# 초기화면
start_date = today[:4] + '-' + today[4:6] + '-' + '01' # today[:4] + '-01-01'
end_date = today[:4] + '-' + today[4:6] + '-' + today[6:]
bidtype = ['con','ser','pur']
for bid in bidtype:
    # 초기화면
    home_url = 'https://infose.info21c.net/info21c/bids/list/index?mode=&page='+str(1)+'&bid_suc=bid&division=1&searchtype=&userid=midoplus&searchWord=&bidtype='+bid+'&g2bcode=&itemcode=&mainforce_code=&keyword=&location=&locationLoc=&whereis=&bid_opt=&contract=&delay=&cancel=&ulevel=&contract_sys=&convention=&sort=-writedt&bid_kind=&word_type=&subWord=&itemcodeAll=&keywordAll=&g2bcodeAll=&whereisAll=&locationAll=&bid_optAll=&conlevel=&cdate_column=writedt&cfrom_date='+start_date+'&cto_date='+end_date+'&pageSize=100'
    driver.get(home_url)

    total_cnt = int(driver.find_element(By.CLASS_NAME, 'list_top').text.split('\n')[0].split(' ')[1].replace(',','')) ## 총 입찰공고건수
    down_page_cnt = (total_cnt // 100) + 1 ## 반복횟수

    bid_info_list = []

    for page_no in range(1,down_page_cnt+1):
        # 웹페이지 접속
        url = 'https://infose.info21c.net/info21c/bids/list/index?mode=&page='+str(page_no)+'&bid_suc=bid&division=1&searchtype=&userid=midoplus&searchWord=&bidtype='+bid+'&g2bcode=&itemcode=&mainforce_code=&keyword=&location=&locationLoc=&whereis=&bid_opt=&contract=&delay=&cancel=&ulevel=&contract_sys=&convention=&sort=-writedt&bid_kind=&word_type=&subWord=&itemcodeAll=&keywordAll=&g2bcodeAll=&whereisAll=&locationAll=&bid_optAll=&conlevel=&cdate_column=writedt&cfrom_date='+start_date+'&cto_date='+end_date+'&pageSize=100'
        driver.get(url)
        time.sleep(1)

        rows = driver.find_elements(By.CSS_SELECTOR, '#w0 > table > tbody > tr') ## 테이블 row 수
        
        for i in range(len(rows)):
            col = rows[i].find_elements(By.TAG_NAME, 'td') ## 테이블 column 수
            cells = []

            for j in range(len(col)):
                cells.append(col[j].text)

            bid_info_list.append(cells)
            
        time.sleep(1)

    # DataFrame으로 변환
    df = pd.DataFrame(bid_info_list)
    df_delete = df.fillna('').apply(lambda x : (x=='').sum(), axis=0) ## 빈칸 컬럼 확인
    df = df[df.columns[df_delete!=df_delete.max()]] ## 빈칸 컬럼 제거

    df.columns = driver.find_elements(By.CSS_SELECTOR, '#w0 > table')[0].text.split('\n')[0].split()

    # 전처리
    df['공고명_re'] = df['공고명/공고번호'].fillna('').str.split('[').str[:-1].apply(('[').join).str.strip().str.split('[')

    df['공고명'] = df['공고명_re'].str[0].str.strip()
    df['분류'] = df['공고명_re'].str[1].str.split().str[1]
    df['공고명'] = np.where(df['분류'].isnull(), df['공고명_re'].str[0].str.strip().str.split().str[:-1].apply(' '.join), df['공고명'])
    df['분류'] = np.where(df['분류'].isnull(), df['공고명_re'].str[0].str.strip().str.split().str[-1], df['분류'])    
    df['공고번호'] = df['공고명/공고번호'].fillna('').str.split('[').str[-1].str.split(']').str[0]
    df['개찰일'] = df['개찰일'].fillna('').str.split('\n').str[0]
    df['개찰일'] = pd.to_datetime(df['개찰일'], format='%y.%m.%d').dt.strftime('%y.%m.%d')
    df['투찰마감'] = df['투찰마감'].fillna('').str.split('\n').str[0]
    df['투찰마감'] = pd.to_datetime(df['투찰마감'], format='%y.%m.%d').dt.strftime('%y.%m.%d')

    df.drop(['공고명/공고번호', '공고명_re'], axis=1, inplace=True)

    bid_info_dict[bid] = df

ValueError: Length mismatch: Expected axis has 6 elements, new values have 7 elements

In [ ]:
# 빅쿼리에 적재
save_dataframe_to_bigquery(bid_info_dict['con'],'info21','bid_con_df_0'+today[4:6],bigquerykey_path)
save_dataframe_to_bigquery(bid_info_dict['ser'],'info21','bid_ser_df_0'+today[4:6],bigquerykey_path)
save_dataframe_to_bigquery(bid_info_dict['pur'],'info21','bid_pur_df_0'+today[4:6],bigquerykey_path)

Data inserted into table bid_con_df_008 successfully.
Data inserted into table bid_ser_df_008 successfully.


KeyError: 'pur'